In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType
from functools import reduce
from operator import add
from pyspark.sql.functions import col

In [3]:
sc = SparkContext(master = 'local[*]')

In [4]:
file = 'data.csv'

In [6]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("PySpark") \
    .getOrCreate()

### 1.Явка (%) по всем регионам

In [7]:
dataframe = spark.read.csv(file, inferSchema=True, header=True).toDF('Region','TIK','UIK', \
          'number of all voters', 'number of ballots received', \
          'number of early voting', 'number of ballots issued inside', 'number of ballots issued outside','number of blanked ballots', \
          'number of ballots in portable boxes', 'number of ballots in stationary boxes', \
          'number of invalid ballots', 'number of valid ballots', 'number of lost ballots', 'number of not counted ballots', \
          'Baburin', 'Grudinin', 'Zhirinovskyi', 'Putin', 'Sobchak', 'Suraikin', 'Titov', 'Yavlisky')

In [8]:
dataframe.show(2)

+--------------------+----------+------+--------------------+--------------------------+----------------------+-------------------------------+--------------------------------+-------------------------+-----------------------------------+-------------------------------------+-------------------------+-----------------------+----------------------+-----------------------------+-------+--------+------------+-----+-------+--------+-----+--------+
|              Region|       TIK|   UIK|number of all voters|number of ballots received|number of early voting|number of ballots issued inside|number of ballots issued outside|number of blanked ballots|number of ballots in portable boxes|number of ballots in stationary boxes|number of invalid ballots|number of valid ballots|number of lost ballots|number of not counted ballots|Baburin|Grudinin|Zhirinovskyi|Putin|Sobchak|Suraikin|Titov|Yavlisky|
+--------------------+----------+------+--------------------+--------------------------+----------------

In [9]:
sum_of_votes = ['Sum of early voting','Sum of inside voting','Sum of outside voting'] #для удобства

In [10]:
#суммируем по необходимым полям
df_result = dataframe.groupBy("Region").agg(F.sum('number of all voters').alias("Sum of all people"), 
                     F.sum('number of early voting').alias("Sum of early voting"), 
                     F.sum('number of ballots issued inside').alias("Sum of inside voting"), 
                     F.sum('number of ballots issued outside').alias("Sum of outside voting")).sort('Sum of all people', ascending=False)
df_result.show(30)

+--------------------+-----------------+-------------------+--------------------+---------------------+
|              Region|Sum of all people|Sum of early voting|Sum of inside voting|Sum of outside voting|
+--------------------+-----------------+-------------------+--------------------+---------------------+
|        город Москва|          7543682|                  0|             4348576|               172779|
|  Московская область|          5829578|                  0|             3540550|               167006|
|  Краснодарский край|          4049575|                229|             2825178|               327900|
|город Санкт-Петер...|          3632398|               1795|             2248088|                70251|
|Свердловская область|          3345221|               3210|             2003456|                79001|
|  Ростовская область|          3210796|                  0|             1920477|               159301|
|Республика Башкор...|          3045698|                  0|    

In [11]:
#считаем процент людей, что проголосовали для каждого региона
final_result = df_result.withColumn("Sum of people voted", 
                                    reduce(add, [col(data) for data in sum_of_votes])) \
            .select('Region', 'Sum of all people','Sum of people voted').sort('Sum of all people')\
            .withColumn("Percents(%)", col('Sum of people voted')/col('Sum of all people'))\
            .select('Region','Percents(%)')\
            .sort('Percents(%)', ascending=False)
final_result.show(30)

+--------------------+------------------+
|              Region|       Percents(%)|
+--------------------+------------------+
|Территория за пре...|0.9806986984380843|
|     Республика Тыва|0.9366369316170003|
|Ямало-Ненецкий ав...|0.9190125747324195|
|Кабардино-Балкарс...|0.9180157863562131|
|Чеченская Республика|0.9154001705101918|
|Республика Северн...|0.8998810071492768|
| Республика Дагестан|0.8747795414462081|
|Карачаево-Черкесс...|0.8740544668203033|
| Кемеровская область|0.8322733516003962|
|Чукотский автоном...|0.8228138695924391|
|Республика Ингушетия|0.8195937165135314|
|    Брянская область| 0.797078003370434|
|   Тюменская область|0.7892544508025854|
|  Краснодарский край|0.7786760339047925|
| Республика Мордовия|0.7785708098460915|
|Республика Татарс...|0.7742157684137118|
|Чувашская Республ...|0.7622296238501433|
|Республика Башкор...|0.7544523455707033|
|  Республика Бурятия|0.7519978782127258|
|Республика Адыгея...|0.7431189118555477|
| Ставропольский край|0.7384811380

### 2.Участок, на котором любимый кандидат получил максимальный результат голосов

In [12]:
favourite_candidat = 'Putin'
n = 300
#UIK = участок
area = dataframe.filter(col(favourite_candidat) > n) \
                .select("Region","TIK","UIK", favourite_candidat)\
                .groupBy("Region","TIK","UIK")\
                .agg(F.max(favourite_candidat).alias('Max amount of votes'))\
                .sort("Max amount of votes",  ascending=False)
area.show(1)        

+--------------------+----+---------+-------------------+
|              Region| TIK|      UIK|Max amount of votes|
+--------------------+----+---------+-------------------+
|Территория за пре...|None|УИК №8342|               7221|
+--------------------+----+---------+-------------------+
only showing top 1 row



In [13]:
favourite_candidat = 'Sobchak'
n = 300
area = dataframe.filter(col(favourite_candidat) > n)\
                .select("Region","TIK","UIK", favourite_candidat)\
                .groupBy("Region","TIK","UIK")\
                .agg(F.max(favourite_candidat).alias('Max amount of votes'))\
                .sort("Max amount of votes",  ascending=False)
area.show(1)        

+--------------------+----+---------+-------------------+
|              Region| TIK|      UIK|Max amount of votes|
+--------------------+----+---------+-------------------+
|Территория за пре...|None|УИК №8061|                868|
+--------------------+----+---------+-------------------+
only showing top 1 row



### 3.Найти регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна 

In [14]:
# 1.находим ТИК с наибольшей явкой = А
# 2.находим ТИК с наименьшея явкой = В
# 3.находим макс (А-В)

tiks = dataframe.groupBy("Region", "TIK")\
            .agg(F.sum('number of all voters').alias("Sum of all people"),\
                     F.sum('number of early voting').alias("Sum of early voting"),\
                     F.sum('number of ballots issued inside').alias("Sum of inside voting"),\
                     F.sum('number of ballots issued outside').alias("Sum of outside voting"))\
            .withColumn('sum of votes', reduce(add, [col(data) for data in sum_of_votes]))\
            .select("Region","TIK","sum of votes")\
            .sort("Region")
tiks.show(10)

+--------------+--------------------+------------+
|        Region|                 TIK|sum of votes|
+--------------+--------------------+------------+
|Алтайский край|        Кытмановская|        7449|
|Алтайский край|     Усть-Калманская|        7430|
|Алтайский край|Новоалтайская гор...|       37312|
|Алтайский край|     Быстроистокская|        4883|
|Алтайский край|        Михайловская|        9724|
|Алтайский край|          Солтонская|        4569|
|Алтайский край|        Ребрихинская|       12023|
|Алтайский край|       Третьяковская|        6950|
|Алтайский край|         Шипуновская|       15066|
|Алтайский край|          Курьинская|        5624|
+--------------+--------------------+------------+
only showing top 10 rows



In [15]:
difference = tiks.groupBy("Region")\
            .agg((F.max('sum of votes') - F.min('sum of votes')).alias("Difference"))\
            .select("Region","Difference") \
            .sort('Difference', ascending=False) 
difference.show(10)

+--------------------+----------+
|              Region|Difference|
+--------------------+----------+
|   Самарская область|    196253|
|Белгородская область|    179081|
| Вологодская область|    170276|
|Ханты-Мансийский ...|    165758|
|  Московская область|    164009|
|  Мурманская область|    153438|
|Ленинградская обл...|    145311|
| Саратовская область|    145021|
|Республика Саха (...|    140985|
|  Краснодарский край|    138053|
+--------------------+----------+
only showing top 10 rows



In [16]:
#просто как вариант найти максимум из полученной таблицы (можно было просто difference.show(1))
result = difference.groupBy("Region")\
        .agg(F.max('Difference').alias("Max difference"))\
        .select("Region","Max difference")\
        .sort("Max difference",ascending=False )
result.show(1)

+-----------------+--------------+
|           Region|Max difference|
+-----------------+--------------+
|Самарская область|        196253|
+-----------------+--------------+
only showing top 1 row



### 4.Посчитать дисперсию по явке для каждого региона (учитывать УИК) 

In [17]:
#суммруем необходимые показатели и считаем дисперсию
disp = dataframe\
        .groupBy("Region","TIK","UIK")\
        .agg(F.sum('number of all voters').alias("Sum of all people"), 
                     F.sum('number of early voting').alias("Sum of early voting"), 
                     F.sum('number of ballots issued inside').alias("Sum of inside voting"), 
                     F.sum('number of ballots issued outside').alias("Sum of outside voting"))\
        .withColumn("Sum of all people", reduce(add, [col(data) for data in sum_of_votes])) \
        .select('Region','TIK', 'UIK','Sum of all people')\
        .groupBy("Region")\
        .agg(F.stddev('Sum of all people').alias("Disp"))\
        .sort("Disp",ascending=False)
        
disp.show(20)

+--------------------+------------------+
|              Region|              Disp|
+--------------------+------------------+
|Территория за пре...|1332.1533610371343|
|Чеченская Республика| 779.5717067527229|
|Ямало-Ненецкий ав...| 739.3290449083053|
|Республика Северн...| 677.9089014204245|
| Республика Дагестан| 657.2845481650245|
|     Республика Тыва| 641.6682099770799|
| Кемеровская область| 621.0871610694816|
| Магаданская область| 614.8118916285795|
|Кабардино-Балкарс...| 614.0460846716693|
|    Брянская область| 592.4564291304056|
|Республика Татарс...|  590.204202181367|
|  Краснодарский край| 584.7406656698317|
|Республика Ингушетия| 583.4284631176162|
| Ставропольский край| 581.8625659348551|
|Ненецкий автономн...| 581.5791490989569|
|Республика Адыгея...| 572.8112314404023|
| Вологодская область| 568.3088067686887|
|   Калужская область| 561.2877773475237|
|    Хабаровский край| 557.9241176416907|
|     Камчатский край| 557.3660784564202|
+--------------------+------------

### 5.Для каждого кандидата посчитать таблицу: результат (%, округленный до целого) количество УИК, на которых кандидат получил данный результат

In [ ]:
# 1.Считаем процент голосов за каждого кандидата относительно всего количества голосов - X
# 2.Считаем процент голосов за каждого кандидата на каждом УИК - Y
# 3.Ищем такие УИК, где X = Y и их количество по каждому президенту

In [18]:
# Группируем по регионам
regions_result = dataframe.groupBy("Region")\
                .agg(F.sum('Baburin').alias('Baburin'),\
                    F.sum('Grudinin').alias('Grudinin'),\
                     F.sum('Zhirinovskyi').alias('Zhirinovskyi'),\
                     F.sum('Putin').alias('Putin'),\
                     F.sum('Sobchak').alias('Sobchak'),\
                     F.sum('Suraikin').alias('Suraikin'),\
                     F.sum('Titov').alias('Titov'),\
                     F.sum('Yavlisky').alias('Yavlisky'))\
                .sort("Region", ascending=True)
regions_result.show(20)

+--------------------+-------+--------+------------+------+-------+--------+-----+--------+
|              Region|Baburin|Grudinin|Zhirinovskyi| Putin|Sobchak|Suraikin|Titov|Yavlisky|
+--------------------+-------+--------+------------+------+-------+--------+-----+--------+
|      Алтайский край|   7581|  281978|       84785|770278|  11788|    7855| 5532|    7259|
|    Амурская область|   2358|   73485|       37909|264493|   4428|    2466| 2080|    1951|
|Архангельская обл...|   4448|   51868|       46925|407190|  10588|    3842| 4982|    6239|
|Астраханская область|   2185|   64047|       19339|342195|   5060|    2823| 2233|    2504|
|Белгородская область|   5218|   93102|       49685|711392|   8474|    6534| 4835|    4445|
|    Брянская область|   4472|   68375|       43940|636087|   7463|    4265| 4175|    3524|
|Владимирская область|   5440|   93649|       58822|546042|  10777|    5075| 6098|    6147|
|Волгоградская обл...|   8040|  140708|       69909|929541|  14403|    8116| 685

In [19]:
# считаем сумму по всем регионам
total = regions_result.agg(F.sum('Baburin').alias('Baburin'),F.sum('Grudinin').alias('Grudinin'),\
                           F.sum('Zhirinovskyi').alias('Zhirinovskyi'),F.sum('Putin').alias('Putin'),\
                           F.sum('Sobchak').alias('Sobchak'),F.sum('Suraikin').alias('Suraikin'),\
                           F.sum('Titov').alias('Titov'),F.sum('Yavlisky').alias('Yavlisky')
                          )
total.show()

+-------+--------+------------+--------+-------+--------+------+--------+
|Baburin|Grudinin|Zhirinovskyi|   Putin|Sobchak|Suraikin| Titov|Yavlisky|
+-------+--------+------------+--------+-------+--------+------+--------+
| 478983| 8658732|     4154638|56426399|1237921|  499287|556751|  769588|
+-------+--------+------------+--------+-------+--------+------+--------+



In [20]:
candidates = ['Baburin','Grudinin','Zhirinovskyi', 'Putin','Sobchak','Suraikin', 'Titov','Yavlisky']

In [21]:
# считаем общее количество
total_amount = total.withColumn('Total', reduce(add, [col(candidat) for candidat in candidates])).collect()
print(total_amount)

[Row(Baburin=478983, Grudinin=8658732, Zhirinovskyi=4154638, Putin=56426399, Sobchak=1237921, Suraikin=499287, Titov=556751, Yavlisky=769588, Total=72782299)]


In [22]:
total = total_amount[0][len(total_amount[0]) - 1]
total_amount = total_amount[0][:(len(total_amount[0])-1)]

In [23]:
total_amount

(478983, 8658732, 4154638, 56426399, 1237921, 499287, 556751, 769588)

In [24]:
#процент голосов за каждого кандидата относительно всего количества голосов - X
X = [round(int(amount)/int(total)*100) for amount in total_amount]
print(X)

[1, 12, 6, 78, 2, 1, 1, 1]


In [25]:

uiks = dataframe.groupBy("Region","UIK")\
                .agg(F.sum('Baburin').alias('Baburin'),\
                    F.sum('Grudinin').alias('Grudinin'),\
                     F.sum('Zhirinovskyi').alias('Zhirinovskyi'),\
                     F.sum('Putin').alias('Putin'),\
                     F.sum('Sobchak').alias('Sobchak'),\
                     F.sum('Suraikin').alias('Suraikin'),\
                     F.sum('Titov').alias('Titov'),\
                     F.sum('Yavlisky').alias('Yavlisky'))\
                .sort("Region", ascending=True)
uiks.show(10)

+--------------+---------+-------+--------+------------+-----+-------+--------+-----+--------+
|        Region|      UIK|Baburin|Grudinin|Zhirinovskyi|Putin|Sobchak|Suraikin|Titov|Yavlisky|
+--------------+---------+-------+--------+------------+-----+-------+--------+-----+--------+
|Алтайский край| УИК №177|     14|     337|          90|  907|     14|      15|    5|       7|
|Алтайский край| УИК №473|      4|     345|          77|  676|      5|       9|    5|       5|
|Алтайский край| УИК №361|      3|     160|          42|  414|      3|       2|    2|       6|
|Алтайский край| УИК №379|     13|     394|          84|  836|     16|      13|   11|      15|
|Алтайский край|УИК №1518|      2|     119|          31|  334|      2|       0|    0|       2|
|Алтайский край|УИК №1314|      1|      16|           8|   72|      0|       1|    0|       0|
|Алтайский край| УИК №925|      1|       9|           2|  102|      1|       0|    0|       2|
|Алтайский край|УИК №1406|      2|      41|       

In [26]:
#считаем общее количество голосов для каждой УИК 
sum_uik = uiks.withColumn('Total', reduce(add, [col(candidat) for candidat in candidates]))
sum_uik.show()

+--------------+---------+-------+--------+------------+-----+-------+--------+-----+--------+-----+
|        Region|      UIK|Baburin|Grudinin|Zhirinovskyi|Putin|Sobchak|Suraikin|Titov|Yavlisky|Total|
+--------------+---------+-------+--------+------------+-----+-------+--------+-----+--------+-----+
|Алтайский край|УИК №1598|      0|       1|          21|   41|      0|       0|    0|       0|   63|
|Алтайский край|УИК №1556|      1|      61|          39|  284|      2|       1|    3|       2|  393|
|Алтайский край| УИК №573|      6|      62|          34|  271|      2|       3|    2|       0|  380|
|Алтайский край| УИК №787|      8|     205|          56|  607|      8|       7|    2|       1|  894|
|Алтайский край|  УИК №74|      6|     249|         106|  682|     12|       8|    5|       9| 1077|
|Алтайский край| УИК №929|      2|      33|          25|  252|      2|       1|    0|       0|  315|
|Алтайский край| УИК №791|      0|      28|          30|  229|      1|       1|    1|      

In [27]:
uik_voting = sum_uik.createOrReplaceTempView('uik_voting')

In [28]:
#для каждого уик считаем процент
percent = spark.sql("""SELECT Region, UIK,
int(round(Baburin / Total * 100)) as Baburin, 
int(round(Grudinin / Total * 100)) as Grudinin,
int(round(Zhirinovskyi / Total * 100)) as Zhirinovskyi, 
int(round(Putin / Total * 100)) as Putin, 
int(round(Sobchak / Total * 100)) as Sobchak, 
int(round(Suraikin / Total * 100)) as Suraikin, 
int(round(Titov / Total * 100)) as Titov, 
int(round(Yavlisky / Total * 100)) as Yavlisky
FROM uik_voting""")

percent.show(10)
percent.createOrReplaceTempView('uik_percent')

+--------------+---------+-------+--------+------------+-----+-------+--------+-----+--------+
|        Region|      UIK|Baburin|Grudinin|Zhirinovskyi|Putin|Sobchak|Suraikin|Titov|Yavlisky|
+--------------+---------+-------+--------+------------+-----+-------+--------+-----+--------+
|Алтайский край| УИК №489|      1|      28|           7|   63|      0|       1|    0|       1|
|Алтайский край|УИК №1271|      0|      26|           8|   63|      0|       2|    2|       0|
|Алтайский край| УИК №550|      1|      23|           7|   66|      1|       1|    1|       1|
|Алтайский край|УИК №1472|      0|      29|          10|   59|      1|       1|    0|       0|
|Алтайский край|  УИК №74|      1|      23|          10|   63|      1|       1|    0|       1|
|Алтайский край|УИК №1556|      0|      16|          10|   72|      1|       0|    1|       1|
|Алтайский край| УИК №636|      0|      25|           5|   67|      1|       0|    1|       0|
|Алтайский край|УИК №1598|      0|       2|       

In [68]:
# 1.Выбираем УИК, где значения совпадают с результатом
baburin_data = spark.sql("""SELECT UIK,Baburin
FROM uik_percent 
WHERE Baburin ="""+str(X[0]))

grudinin_data = spark.sql("""SELECT UIK,Grudinin
FROM uik_percent 
WHERE Grudinin = """+str(X[1]))

zhirinovskyi_data = spark.sql("""SELECT UIK,Zhirinovskyi
FROM uik_percent 
WHERE Zhirinovskyi = """+str(X[2]))

putin_data = spark.sql("""SELECT UIK,Putin
FROM uik_percent 
WHERE Putin = """+str(X[3]))

sobchak_data = spark.sql("""SELECT UIK,Sobchak
FROM uik_percent 
WHERE Sobchak = """+str(X[4]))

suraikin_data = spark.sql("""SELECT UIK,Suraikin
FROM uik_percent 
WHERE Suraikin = """+str(X[5]))

titov_data = spark.sql("""SELECT UIK,Titov
FROM uik_percent 
WHERE  Titov = """+str(X[6]))

yavlisky_data = spark.sql("""SELECT UIK,Yavlisky
FROM uik_percent 
WHERE Yavlisky = """+str(X[7]))

In [69]:
baburin_data.show()
baburin_data.createOrReplaceTempView('baburin_data')

+---------+-------+
|      UIK|Baburin|
+---------+-------+
|УИК №1314|      1|
| УИК №169|      1|
| УИК №824|      1|
| УИК №859|      1|
| УИК №490|      1|
| УИК №922|      1|
| УИК №211|      1|
|УИК №1704|      1|
|УИК №1758|      1|
| УИК №206|      1|
| УИК №727|      1|
|УИК №1399|      1|
| УИК №609|      1|
| УИК №709|      1|
| УИК №176|      1|
|УИК №1145|      1|
| УИК №333|      1|
| УИК №705|      1|
|УИК №1292|      1|
|УИК №1192|      1|
+---------+-------+
only showing top 20 rows



In [70]:
grudinin_data.show()
grudinin_data.createOrReplaceTempView('grudinin_data')

+---------+--------+
|      UIK|Grudinin|
+---------+--------+
|УИК №1054|      12|
|УИК №1576|      12|
|УИК №1012|      12|
| УИК №823|      12|
| УИК №836|      12|
|УИК №1096|      12|
|УИК №1487|      12|
| УИК №854|      12|
|УИК №1274|      12|
|УИК №1272|      12|
| УИК №795|      12|
|УИК №1703|      12|
|УИК №1686|      12|
|УИК №1130|      12|
|УИК №1653|      12|
| УИК №558|      12|
|УИК №1267|      12|
|УИК №1040|      12|
|УИК №1278|      12|
| УИК №537|      12|
+---------+--------+
only showing top 20 rows



In [71]:
zhirinovskyi_data.show()
zhirinovskyi_data.createOrReplaceTempView('zhirinovskyi_data')

+---------+------------+
|      UIK|Zhirinovskyi|
+---------+------------+
| УИК №367|           6|
|УИК №1145|           6|
|УИК №1030|           6|
| УИК №468|           6|
|УИК №1647|           6|
|УИК №1214|           6|
| УИК №295|           6|
| УИК №177|           6|
|УИК №1316|           6|
| УИК №176|           6|
|УИК №1359|           6|
| УИК №982|           6|
|УИК №1518|           6|
|УИК №1137|           6|
| УИК №379|           6|
|УИК №1292|           6|
| УИК №787|           6|
| УИК №169|           6|
| УИК №167|           6|
| УИК №314|           6|
+---------+------------+
only showing top 20 rows



In [72]:
putin_data.show()
putin_data.createOrReplaceTempView('putin_data')

+---------+-----+
|      UIK|Putin|
+---------+-----+
|УИК №1717|   78|
| УИК №966|   78|
|УИК №1296|   78|
| УИК №590|   78|
|УИК №1546|   78|
|УИК №1576|   78|
| УИК №917|   78|
| УИК №854|   78|
| УИК №893|   78|
|УИК №1650|   78|
| УИК №884|   78|
| УИК №584|   78|
|УИК №1455|   78|
| УИК №959|   78|
| УИК №970|   78|
| УИК №832|   78|
| УИК №808|   78|
|УИК №1487|   78|
|УИК №1040|   78|
|УИК №1060|   78|
+---------+-----+
only showing top 20 rows



In [73]:
sobchak_data.show()
sobchak_data.createOrReplaceTempView('sobchak_data')

+---------+-------+
|      UIK|Sobchak|
+---------+-------+
|УИК №1130|      2|
| УИК №147|      2|
| УИК №171|      2|
| УИК №295|      2|
| УИК №919|      2|
| УИК №199|      2|
|УИК №1826|      2|
| УИК №992|      2|
| УИК №354|      2|
|УИК №1137|      2|
|УИК №1541|      2|
| УИК №217|      2|
| УИК №132|      2|
| УИК №239|      2|
| УИК №167|      2|
| УИК №115|      2|
|УИК №1075|      2|
| УИК №817|      2|
|УИК №1883|      2|
| УИК №614|      2|
+---------+-------+
only showing top 20 rows



In [74]:
suraikin_data.show()
suraikin_data.createOrReplaceTempView('suraikin_data')

+---------+--------+
|      UIK|Suraikin|
+---------+--------+
|УИК №1220|       1|
| УИК №593|       1|
| УИК №211|       1|
|УИК №1898|       1|
| УИК №177|       1|
| УИК №727|       1|
|УИК №1647|       1|
|  УИК №74|       1|
| УИК №167|       1|
| УИК №787|       1|
| УИК №489|       1|
|УИК №1017|       1|
|УИК №1472|       1|
|УИК №1075|       1|
| УИК №115|       1|
|УИК №1221|       1|
| УИК №573|       1|
| УИК №562|       1|
| УИК №550|       1|
| УИК №132|       1|
+---------+--------+
only showing top 20 rows



In [75]:
titov_data.show()
titov_data.createOrReplaceTempView('titov_data')

+---------+-----+
|      UIK|Titov|
+---------+-----+
|УИК №1904|    1|
| УИК №479|    1|
| УИК №239|    1|
| УИК №169|    1|
|УИК №1455|    1|
| УИК №167|    1|
| УИК №468|    1|
| УИК №573|    1|
| УИК №982|    1|
|УИК №1085|    1|
| УИК №636|    1|
| УИК №211|    1|
| УИК №569|    1|
| УИК №379|    1|
|УИК №1556|    1|
| УИК №176|    1|
| УИК №550|    1|
| УИК №520|    1|
|УИК №1196|    1|
| УИК №131|    1|
+---------+-----+
only showing top 20 rows



In [76]:
yavlisky_data.show()
yavlisky_data.createOrReplaceTempView('yavlisky_data')

+---------+--------+
|      UIK|Yavlisky|
+---------+--------+
| УИК №468|       1|
|УИК №1145|       1|
| УИК №132|       1|
| УИК №489|       1|
| УИК №167|       1|
| УИК №569|       1|
|УИК №1556|       1|
| УИК №922|       1|
| УИК №211|       1|
|УИК №1196|       1|
| УИК №616|       1|
| УИК №177|       1|
| УИК №115|       1|
| УИК №550|       1|
|  УИК №74|       1|
| УИК №520|       1|
| УИК №131|       1|
| УИК №176|       1|
| УИК №727|       1|
| УИК №724|       1|
+---------+--------+
only showing top 20 rows



In [86]:
#считаем их количество
baburin = spark.sql("""SELECT count(UIK)
FROM baburin_data""").collect()
grudinin = spark.sql("""SELECT count(UIK)
FROM grudinin_data""").collect()
zhirinovskyi = spark.sql("""SELECT count(UIK)
FROM zhirinovskyi_data""").collect()
putin = spark.sql("""SELECT count(UIK)
FROM putin_data""").collect()
sobchak = spark.sql("""SELECT count(UIK)
FROM sobchak_data""").collect()
suraikin = spark.sql("""SELECT count(UIK)
FROM suraikin_data""").collect()
yavlisky = spark.sql("""SELECT count(UIK)
FROM yavlisky_data""").collect()
titov = spark.sql("""SELECT count(UIK) FROM titov_data""").collect()


In [107]:
baburin = baburin[0][0]
grudinin = grudinin[0][0]
zhirinovskyi = zhirinovskyi[0][0]
putin = putin[0][0]
sobchak = sobchak[0][0]
suraikin = suraikin[0][0]
yavlisky = yavlisky[0][0]
titov = titov[0][0]

In [124]:
columns = ["Title","Baburin","Grudinin","Zhirinovskyi","Putin","Sobchak","Suraikin","Yavlisky","Titov"]
data = [('Result (в %)',X[0],X[1],X[2],X[3],X[4],X[5],X[6],X[7]),('UIK count',baburin,grudinin,zhirinovskyi,putin,sobchak,suraikin,yavlisky,titov)]

In [125]:
#формируем итоговую таблицу
result = spark.createDataFrame(data, columns)
result.show()

+------------+-------+--------+------------+-----+-------+--------+--------+-----+
|       Title|Baburin|Grudinin|Zhirinovskyi|Putin|Sobchak|Suraikin|Yavlisky|Titov|
+------------+-------+--------+------------+-----+-------+--------+--------+-----+
|Result (в %)|      1|      12|           6|   78|      2|       1|       1|    1|
|   UIK count|  48189|    7798|       13590| 4781|  18667|   49149|   30892|40154|
+------------+-------+--------+------------+-----+-------+--------+--------+-----+

